In [ ]:
!pip install mlflow boto3 awscli

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 84.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 570.5/570.5 kB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.0/677.0 kB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20

In [ ]:
from google.colab import userdata
import os

# Retrieve AWS credentials from Colab Secrets
aws_access_key = userdata.get('AWS_ACCESS_KEY_ID')
aws_secret_key = userdata.get('AWS_SECRET_ACCESS_KEY')
mlflow_uri = userdata.get('MLFLOW_TRACKING_URI')

# Set as environment variables (for boto3 and AWS CLI)
os.environ["AWS_ACCESS_KEY_ID"] = aws_access_key
os.environ["AWS_SECRET_ACCESS_KEY"] = aws_secret_key

In [ ]:
import mlflow
mlflow.set_tracking_uri(mlflow_uri)
mlflow.set_experiment("Exp 2 - BoW vs TfIdf")

2025/07/24 22:39:05 INFO mlflow.tracking.fluent: Experiment with name 'Exp 2 - BoW vs TfIdf' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://sumanthdatahub/mlflow-artifacts/586640530243144839', creation_time=1753396745192, experiment_id='586640530243144839', last_update_time=1753396745192, lifecycle_stage='active', name='Exp 2 - BoW vs TfIdf', tags={}>

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Read Preprocesssing Data
df = pd.read_csv('/content/reddit_preprocessing.csv').dropna(subset=['clean_comment'])
df.shape

(36710, 6)

In [ ]:
def run_experiment(vectorizer_type, ngram_range, vectorizer_max_features, vectorizer_name):
  # Vectorization
  if vectorizer_type == 'BoW':
    vectorizer = CountVectorizer(ngram_range=ngram_range, max_features=vectorizer_max_features)
  else:
    vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=vectorizer_max_features)

  X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2, random_state=42)

  X_train_vectorized = vectorizer.fit_transform(X_train)
  X_test_vectorized = vectorizer.transform(X_test)

  # Define and Train RF model
  with mlflow.start_run() as run:
    # Set Tags
    tags = {
        "mlflow.runName": f"{vectorizer_type}_{vectorizer_name}_RandomForest",
        "experiment_type": "feature_engineering",
        "model_type": "RandomForestClassifier",
        "description": f"RandomForest with {vectorizer_name}, ngram_range={ngram_range}, max_features={vectorizer_max_features}",

    }
    mlflow.set_tags(tags)

    # Set Params
    n_estimators = 200
    max_depth = 15

    params = {
        "vectorizer_type": vectorizer_type,
        "ngram_range": ngram_range,
        "vectorizer_max_features": vectorizer_max_features,
        "n_estimators": n_estimators,
        "max_depth": max_depth
    }
    mlflow.log_params(params)

    # Initialize and Train the Model
    rf_model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
    rf_model.fit(X_train_vectorized, y_train)

    # Make Predictions and Log Metrics
    y_pred = rf_model.predict(X_test_vectorized)

    # Log Accuracy
    accuracy = accuracy_score(y_test, y_pred)
    mlflow.log_metric("accuracy", accuracy)

    # Log Classification Report
    classification_metrics = classification_report(y_test, y_pred, output_dict=True)
    for class_label, metrics in classification_metrics.items():
        if isinstance(metrics, dict):
            for metric_name, metric_value in metrics.items():
                mlflow.log_metric(f"{class_label}_{metric_name}", metric_value)

    # Log Confusion Matrix
    conf_matrix = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title(f'Confusion Matrix: {vectorizer_name}, {ngram_range}')
    plt.savefig('confusion_matrix.png')
    mlflow.log_artifact('confusion_matrix.png')
    plt.close()

    # Log the Model
    mlflow.sklearn.log_model(rf_model, f"random_forest_model_{vectorizer_name}_{ngram_range}")

# Run Experiments for BoW and TF-IDF with different n-grams
ngram_ranges = [(1, 1), (1, 2), (1, 3)] # unigrams, bigrams, trigrams
max_features_values = 5000 # Example max feature size
vectorizer_types = ['BoW', 'TfIdf']

for vectorizer_type in vectorizer_types:
    for ngram_range in ngram_ranges:
        run_experiment(vectorizer_type, ngram_range, max_features_values, vectorizer_type)


2025/07/24 23:06:22 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/24 23:06:31 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run BoW_BoW_RandomForest at: http://ec2-54-87-129-244.compute-1.amazonaws.com:5000/#/experiments/586640530243144839/runs/64a3ec411d5842e9983a419f23f963c0
🧪 View experiment at: http://ec2-54-87-129-244.compute-1.amazonaws.com:5000/#/experiments/586640530243144839


2025/07/24 23:06:46 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/24 23:06:51 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run BoW_BoW_RandomForest at: http://ec2-54-87-129-244.compute-1.amazonaws.com:5000/#/experiments/586640530243144839/runs/0d6c25ac7e7e4d5fa1d4a1486b4f5e22
🧪 View experiment at: http://ec2-54-87-129-244.compute-1.amazonaws.com:5000/#/experiments/586640530243144839


2025/07/24 23:07:09 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/24 23:07:14 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

🏃 View run TfIdf_TfIdf_RandomForest at: http://ec2-54-87-129-244.compute-1.amazonaws.com:5000/#/experiments/586640530243144839/runs/768df11cdb7c4599867967b310cf94b6
🧪 View experiment at: http://ec2-54-87-129-244.compute-1.amazonaws.com:5000/#/experiments/586640530243144839


2025/07/24 23:07:48 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/24 23:07:52 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TfIdf_TfIdf_RandomForest at: http://ec2-54-87-129-244.compute-1.amazonaws.com:5000/#/experiments/586640530243144839/runs/39d4db074f7b4ea497e785f536820132
🧪 View experiment at: http://ec2-54-87-129-244.compute-1.amazonaws.com:5000/#/experiments/586640530243144839


2025/07/24 23:08:13 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/24 23:08:18 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TfIdf_TfIdf_RandomForest at: http://ec2-54-87-129-244.compute-1.amazonaws.com:5000/#/experiments/586640530243144839/runs/ea21bc86f7c746c9b90510f3f321ed5b
🧪 View experiment at: http://ec2-54-87-129-244.compute-1.amazonaws.com:5000/#/experiments/586640530243144839


Decided to go with TF IDF, (1,3), MLFLOW Comparision shows High Recall for -1

We try to find the best Max Features